In [40]:
import pandas as pd
import spacy
import pprint as pprint
import numpy as np
import os
import random
import math
import time
from operator import itemgetter
import random

In [41]:
nlp = spacy.load("en")

In [42]:
train_data = np.load("processed_data/test_data.npy") #change path based on requirement

In [43]:
mismatch_tokenization = []
mismatch_subtokenization = []
passage_id_to_id_dict = {}
id_to_passageid_dict = {}
valid_id_to_original_id_dict = {}
train_sent_len_list = []
train_passage_list = [] 
train_tokens_list = []
train_embeddings_list = []
train_sentence_cum_lengths = []
valid_train_docs = 0
train_exception_passages = [886, 1990, 2709, 2710, 1687,2200, 2724]
negative_samples_factor = 5
train_positive_pairs = []
train_negative_pairs = []

In [44]:
for i, doc in enumerate(train_data):
    
    if(i in train_exception_passages):
        continue
    if(i%100 == 0):
        print("processed docs = ", i)
    sentences = doc[0]
    tokens = doc[1]
    speakers = doc[3]
    doc_id = doc[4]
    part_id = doc[5]
    total_tokens = []
    passage_text = ""
    included_setences = 0
    sent_lens = []
    previous_length = 0
    #get passage text and tokens of 512 length
    for sent in sentences:
        if(len((passage_text + sent).split()) >= 400):
            break
        included_setences += 1
        passage_text += sent
    for token_list in tokens[0:included_setences]:
        sent_lens.append(previous_length)
        previous_length += len(token_list)
        total_tokens += token_list
    
    #remove mismatch tokens
    valid = True
    passage_text_processed = nlp(passage_text)
    spacy_tokens = []
    for w_i, word in enumerate(passage_text_processed):
        spacy_tokens += [word]
    if(len(spacy_tokens) != len(total_tokens)):
        mismatch_tokenization += [i]
        valid = False
        continue
    else:
        for (a,b) in zip(total_tokens, spacy_tokens):
            if(a.lower() != b.text.lower()): #check at individual tokens level
                print("mismatch of ", a.lower(), b.text.lower())
                mismatch_subtokenization += [i]
                valid = False
                break
    
    
    #save valid tokensmatc
    if(valid):
        #bert_embeddings = []
        #bert_embeddings = get_bert_embeddings(passage_text)
        #assert len(bert_embeddings) == len(total_tokens)
        passage_id_to_id_dict.update({doc_id+part_id:valid_train_docs})
        id_to_passageid_dict.update({valid_train_docs:doc_id+part_id})
        train_sent_len_list.append(included_setences)
        train_passage_list.append(passage_text)
        train_tokens_list.append(total_tokens)
        #train_embeddings_list.append(bert_embeddings)
        train_sentence_cum_lengths.append(sent_lens)
        valid_id_to_original_id_dict.update({valid_train_docs:i})
        valid_train_docs += 1

processed docs =  0
processed docs =  100
processed docs =  200
processed docs =  300


In [45]:
skip_passages = len(mismatch_tokenization) + len(train_exception_passages)
valid_passages = len(id_to_passageid_dict)
print(valid_passages, skip_passages, valid_passages + skip_passages)

133 222 355


In [46]:
def parse_corefs():
    train_coref_list = []
    skipped = 0
    for i in range(valid_passages):
        valid_sentences = train_sent_len_list[i]
        tokens = train_tokens_list[i]
        cumulative_lengths = train_sentence_cum_lengths[i]
        original_id = valid_id_to_original_id_dict[i]
        corefs_list = train_data[original_id][2][0:valid_sentences]
        doc_coref_dict = {}
        for (j, sent_corefs_list) in enumerate(corefs_list):
            for coref_dicts in sent_corefs_list:
                start = int(coref_dicts['start']) + cumulative_lengths[j]
                end = int(coref_dicts['end']) + cumulative_lengths[j]
                cluster_id = int(coref_dicts['label'])
                if start >= len(tokens) or end >= len(tokens):
                    skipped += 1
                else:
                    if cluster_id in doc_coref_dict.keys():
                        doc_coref_dict[cluster_id].append((start, end))
                    else:
                        doc_coref_dict.update({cluster_id:[(start, end)]})
        train_coref_list.append(doc_coref_dict)
    return train_coref_list

In [47]:
train_coref_list = parse_corefs()

In [48]:
for i, doc_coref_list in enumerate(train_coref_list):
    for key, value in doc_coref_list.items():
        train_coref_list[i][key] = sorted(value, key = lambda x: x[0]) 

In [49]:
for (i,doc_coref_list) in enumerate(train_coref_list):
    coref_list = []
    cluster_ids = list(doc_coref_list.keys())
    positive_pairs = []
    negative_pairs = []
    #print("number of clusters", len(cluster_ids))
    for key, value in doc_coref_list.items():
        for i in range(0, len(doc_coref_list[key])):
            for j in range(i+1, len(doc_coref_list[key])):
                positive_pairs.append((doc_coref_list[key][i], doc_coref_list[key][j]))
    #print("number of positive samples", len(positive_pairs))
    negative_samples = len(positive_pairs) * negative_samples_factor
    while len(negative_pairs) < negative_samples and len(cluster_ids) > 1:
        t_menA_id = cluster_ids[random.randint(0, len(cluster_ids))-1]
        t_menB_id = t_menA_id
        while t_menB_id == t_menA_id:
            t_menB_id = cluster_ids[random.randint(0, len(cluster_ids))-1]
        menA_id = min(t_menA_id, t_menB_id)
        menB_id = max(t_menA_id, t_menB_id)
        a_index = random.randint(0, len(doc_coref_list[menA_id])-1)
        b_index = random.randint(0, len(doc_coref_list[menB_id])-1)
        menA_id = doc_coref_list[menA_id][a_index]
        menB_id = doc_coref_list[menB_id][b_index]
        negative_pairs.append((menA_id, menB_id))
    train_positive_pairs.append(positive_pairs)
    train_negative_pairs.append(negative_pairs)

In [50]:
# np.save("processed_data/test_positive_pairs.npy", train_positive_pairs)
np.save("processed_data/small_test_negative_pairs.npy", train_negative_pairs)
# np.save("processed_data/test_passage_list.npy", train_passage_list)
# np.save("processed_data/test_valid_id_to_original_id_dict.npy", valid_id_to_original_id_dict)
# np.save("processed_data/test_coref_list.npy", train_coref_list)